In [ ]:
# --- Install missing packages if necessary ---
# !pip install requests geopandas shapely

import os

import geopandas as gpd
import requests
from shapely.geometry import LineString, MultiPolygon, Point, Polygon

# --- User input ---
print("Enter the country (ISO3166-1 code, e.g., DE, FR, AT):")
country = input().strip().upper()

# --- Overpass API endpoint ---
OVERPASS_URL = "https://overpass-api.de/api/interpreter"

# --- Overpass QL query: fetch all wildlife crossings with geometry ---
query = f"""
[out:json][timeout:1800];
area["ISO3166-1"="{country}"][admin_level=2]->.searchArea;
(
  node["man_made"="wildlife_crossing"](area.searchArea);
  way["man_made"="wildlife_crossing"](area.searchArea);
  relation["man_made"="wildlife_crossing"](area.searchArea);
);
out body geom;
"""

# --- Send request ---
response = requests.get(OVERPASS_URL, params={"data": query})
response.raise_for_status()
data = response.json()

# --- Extract elements ---
elements = data["elements"]

# --- Build GeoDataFrame ---
features = []

for el in elements:
    tags = el.get("tags", {})

    # Node → Point
    if el["type"] == "node":
        geometry = Point(el["lon"], el["lat"])
        features.append({"id": el["id"], "geometry": geometry, **tags})

    # Way → LineString or Polygon
    elif el["type"] == "way" and "geometry" in el:
        coords = [(pt["lon"], pt["lat"]) for pt in el["geometry"]]
        if len(coords) >= 3 and coords[0] == coords[-1]:
            geometry = Polygon(coords)  # Closed → Polygon
        else:
            geometry = LineString(coords)  # Open → LineString
        features.append({"id": el["id"], "geometry": geometry, **tags})

    # Relation → MultiPolygon (if members have geometry)
    elif el["type"] == "relation" and "members" in el:
        polygons = []
        for member in el["members"]:
            if member["type"] == "way" and "geometry" in member:
                coords = [(pt["lon"], pt["lat"]) for pt in member["geometry"]]
                if len(coords) >= 3:
                    polygons.append(Polygon(coords))
        if polygons:
            geometry = MultiPolygon(polygons) if len(polygons) > 1 else polygons[0]
            features.append({"id": el["id"], "geometry": geometry, **tags})

# --- Create GeoDataFrame ---
if features:
    gdf = gpd.GeoDataFrame(features, geometry="geometry", crs="EPSG:4326")
else:
    gdf = gpd.GeoDataFrame(columns=["id", "geometry"])

# --- Filter by bridge = 'yes' if needed ---
if "bridge" in gdf.columns:
    gdf = gdf[gdf["bridge"] == "yes"]

# --- Reproject to EPSG:3035 (ETRS89 / LAEA Europe) ---
if not gdf.empty:
    gdf = gdf.to_crs(epsg=3035)
    print("Reprojected to EPSG:3035.")

# --- Save GeoJSON ---
output_dir = os.path.abspath(os.path.join(os.getcwd(), "..", "data"))
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, f"{country}_wildlife_crossings.geojson")
gdf.to_file(output_file, driver="GeoJSON")

print(f"GeoDataFrame contains {len(gdf)} features after filtering.")
print(f"Saved to {output_file}")
gdf.head()


Enter the country (ISO3166-1 code, e.g., DE, FR, AT):


 DE


Reprojected to EPSG:3035.
GeoDataFrame contains 131 features after filtering.
Saved to C:\tmp\CAS Spatial Data Analytics\Dokumentation\Projektarbeit\Datenverarbeitung\CASSDA_py\cassda-zertifikatsarbeit\data\DE_wildlife_crossings.geojson


,id,geometry,description,man_made,tunnel,material,name,layer,bridge,bridge:structure,...,construction,informal,mapillary,abandoned:highway,wikipedia,fence_type,wikimedia_commons,intermittent,fixme,type
6,32784047,"POLYGON ((4366794.638 3238923.789, 4366828.877...",Wildbrücke,wildlife_crossing,NaN,NaN,NaN,1,yes,beam,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,32847260,"POLYGON ((4421932.777 3402062.2, 4421911.955 3...",NaN,wildlife_crossing,NaN,NaN,Grünbrücke Ahrensboek,1,yes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,34301551,"POLYGON ((4369678.151 3241900.73, 4369676.746 ...",Wildbrücke,wildlife_crossing,NaN,NaN,NaN,1,yes,beam,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,60131669,"LINESTRING (4613596.527 3119257.356, 4613615.4...",NaN,wildlife_crossing,NaN,NaN,Haselmausbrücke,1,yes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,65243312,"POLYGON ((4337328.921 3432107.197, 4337327.16 ...",NaN,wildlife_crossing,NaN,NaN,Grünbrücke Kiebitzholm,1,yes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# --- Install missing packages if necessary ---
# !pip install requests geopandas shapely pyproj

import os

import geopandas as gpd
import requests
from shapely.geometry import LineString

# --- User input for country ---
print("Please enter the country (ISO3166-1 code, e.g., DE, FR, AT):")
country = input().strip().upper()

# --- Overpass API endpoint ---
OVERPASS_URL = "https://overpass-api.de/api/interpreter"

# --- Overpass API query: motorways, trunks, and primaries ---
query = f"""
[out:json][timeout:1800];
area["ISO3166-1"="{country}"][admin_level=2]->.searchArea;
(
  way["highway"="motorway"](area.searchArea);
  way["highway"="trunk"](area.searchArea);
  way["highway"="primary"](area.searchArea);
);
out body geom;
>;
out skel qt;
"""

# --- Send request to Overpass API ---
response = requests.get(OVERPASS_URL, params={"data": query})
response.raise_for_status()
data = response.json()

# --- Extract elements ---
elements = data["elements"]

# --- Convert to GeoDataFrame (only LineStrings) ---
features = []
for el in elements:
    if el["type"] == "way" and "geometry" in el:
        coords = [(pt["lon"], pt["lat"]) for pt in el["geometry"]]
        # Only open ways → LineString
        if len(coords) >= 2 and coords[0] != coords[-1]:
            features.append(
                {"id": el["id"], "geometry": LineString(coords), **el.get("tags", {})}
            )

# --- Build GeoDataFrame ---
if not features:
    print("No LineString features found!")
    gdf = gpd.GeoDataFrame(columns=["id", "geometry"])
else:
    gdf = gpd.GeoDataFrame(features, geometry="geometry", crs="EPSG:4326")
    print(f"Found {len(gdf)} LineString features.")

# --- Prepare output folder ---
output_dir = os.path.abspath(os.path.join(os.getcwd(), "..", "data"))
os.makedirs(output_dir, exist_ok=True)

# --- Reproject to EPSG:3035 (ETRS89 / LAEA Europe) ---
if not gdf.empty:
    gdf = gdf.to_crs(epsg=3035)
    print("Reprojected to EPSG:3035.")

# --- Save original GeoJSON ---
output_file = os.path.join(
    output_dir, f"{country}_highway_motorway_trunk_primary.geojson"
)
gdf.to_file(output_file, driver="GeoJSON")
print(f"Data saved to {output_file}")

# --- Create 150m buffer ---
if not gdf.empty:
    gdf_buffered = gdf.copy()
    gdf_buffered["geometry"] = gdf.buffer(150)  # Buffer 150 meters

    # Save buffered GeoJSON
    buffer_file = os.path.join(
        output_dir, f"{country}_highway_motorway_trunk_primary_buffer150m.geojson"
    )
    gdf_buffered.to_file(buffer_file, driver="GeoJSON")
    print(f"Buffered data saved to {buffer_file}")
else:
    print("No data to buffer.")

# --- Show preview ---
gdf.head()

Please enter the country (ISO3166-1 code, e.g., DE, FR, AT):
Found 44210 LineString features.
Reprojected to EPSG:3035.
Data saved to c:\code\cassda-zertifikatsarbeit\data\CH_highway_motorway_trunk_primary.geojson
Buffered data saved to c:\code\cassda-zertifikatsarbeit\data\CH_highway_motorway_trunk_primary_buffer150m.geojson


,id,geometry,cycleway:right,destination,highway,lanes,maxspeed,oneway,ref,surface,...,toll:N2,name:rm,addr:city,oneway:vehicle,man_made,disused:maxspeed,motorcar:lanes:backward,cycleway:left:width,change:bus:lanes,parking:both:orientation
0,1344275,"LINESTRING (4155165.731 2619544.353, 4155191.8...",lane,Interlaken-West;Beatenberg;Habkern,primary,1,80,yes,6;11,asphalt,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1344276,"LINESTRING (4155426.629 2619724.622, 4155418.1...",lane,NaN,primary,1,70,yes,6;11,asphalt,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3728938,"LINESTRING (4130459.092 2659391.767, 4130482.0...",NaN,NaN,primary,NaN,50,NaN,1;12,asphalt,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4006662,"LINESTRING (4126158.863 2652100.125, 4126154.1...",lane,Zürich;Basel;Biel,primary,2,50,yes,1;12,asphalt,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4019615,"LINESTRING (4231304.771 2710311.805, 4231347.4...",lane,NaN,primary,2,60,NaN,7,asphalt,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
